<a href="https://colab.research.google.com/github/javahedi/project-On-GoogleColab/blob/main/cnn_MNIST_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifying MNIST handwriting digits witn CNN

In [ ]:
# loading the dataset
import torch
from torch.utils.data import Subset
import torchvision 
from torchvision import transforms

image_path = '/content/MNIST'
# custom trnasformation: converting 
# PIXEL to FLOAT type in the form of tensor
# and also normalize pixels from [0,255] to [0,1]
transform = transforms.Compose([transforms.ToTensor()])


mnist_datset = torchvision.datasets.MNIST(
    root=image_path, train = True,
    transform = transform, download=True
)

mnist_train_datset = Subset(mnist_datset, 
                            torch.arange(10000,))

mnist_valid_datset = Subset(mnist_datset, 
                            torch.arange(10000, len(mnist_datset)))

mnist_test_datset = torchvision.datasets.MNIST(
    root=image_path, train = False,
    transform = transform, download=True
)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /content/MNIST/MNIST/raw/train-images-idx3-ubyte.gz to /content/MNIST/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /content/MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to /content/MNIST/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /content/MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to /content/MNIST/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /content/MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/MNIST/MNIST/raw



In [ ]:
from torch.utils.data.dataloader import DataLoader
batch_size = 64
torch.manual_seed(1)

train_dl = DataLoader(mnist_train_datset, batch_size, shuffle=True)
valid_dl = DataLoader(mnist_valid_datset, batch_size, shuffle=True)

## constructing a CNN in PyTorch

In [ ]:
import torch.nn as nn

model = nn.Sequential()

model.add_module(
    'conv1', nn.Conv2d(
        in_channels=1, out_channels=32,
        kernel_size=5, padding=2
    )
)
model.add_module('relu1', nn.ReLU())
model.add_module('pool1', nn.MaxPool2d(kernel_size=2))

model.add_module(
    'conv2', nn.Conv2d(
        in_channels=32, out_channels=64,
        kernel_size=5, padding=2
    )
)
model.add_module('relu2', nn.ReLU())
model.add_module('po0l2', nn.MaxPool2d(kernel_size=2))


- While we can calculate the size of the feature maps at this stage manually, 
- PyTorch provides a convenient method to compute this for us:

In [ ]:
# an example , input to the model, batch_size=4, channel =1, (spatial)image_size=28*28
x = torch.ones((4,1,28,28)) # 
model(x).shape

torch.Size([4, 64, 7, 7])

mode output is batch_size=4, (feature maps)channel =64, (spatial)image_size=7*7


- newt, we add a fully conenctec layer, for classification, on top of CNN layers and poolings
- the input of this layer must have **rank-2**

In [ ]:
model.add_module('flatten', nn.Flatten())
x = torch.ones((4,1,28,28)) # 
model(x).shape # 7*7*64

torch.Size([4, 3136])

## Dropout:
 - a popular technique fo regularization, (like "L1", Lasso), to avoid overfitting in deep NN
 - note, dropout is just implemented on **training** process not in the **evaluation** process

In [ ]:
model.add_module('fc1', nn.Linear(3136, 1024))
model.add_module('relu3', nn.ReLU())
model.add_module('dropout', nn.Dropout(p=0.5))
model.add_module('fc2', nn.Linear(1024, 10)) # needs to classifing 10 classes

 - for multi-classification problem, one usually, use **softmat** activation to obtain class-membership probability
 - However, **softmax** function is already used internally inside PyTorch's ***CrossEntropyLoss*

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

## **Adam**:
- is robout optimizer, gradient-based optimization method suited to **nonconvex** optimization and ML problems.
- update step size derived fropm the running average of gradient moments.


Now, let's train out model

In [ ]:
def train(model, num_epoches, train_dl, valid_dl):
    loss_hist_train = [0] * num_epoches
    loss_hist_valid = [0] * num_epoches
    accu_hist_train = [0] * num_epoches
    accu_hist_valid = [0] * num_epoches

    for epoch in range(num_epoches):
        model.train()
        for x_batch, y_batch in train_dl:
            pred = model(x_batch)
            loss = loss_fn(pred, y_batch)
            loss.backward()
            optimizer.step() 
            optimizer.zero_grad()
            loss_hist_train[epoch] += loss.item() * y_batch.size(0)
            is_correct = (torch.argmax(pred, dim = 1) == y_batch).float()
            accu_hist_train[epoch] += is_correct.sum()
        loss_hist_train[epoch] /= len(train_dl.dataset)
        accu_hist_train[epoch] /= len(train_dl.dataset)

        model.eval()
        with torch.no_grad():
            for x_batch, y_batch in valid_dl:
                pred = model(x_batch)
                loss = loss_fn(pred, y_batch)
                loss_hist_valid[epoch] += loss.item() * y_batch.size(0)
                is_correct = (torch.argmax(pred, dim = 1) == y_batch).float()
                accu_hist_valid[epoch] += is_correct.sum()
        loss_hist_valid[epoch] /= len(valid_dl.dataset)
        accu_hist_valid[epoch] /= len(valid_dl.dataset)


        print(f'Epoch {epoch+1:3d}  train_accuracy: '
              f'{accu_hist_train[epoch]:.4f}, val_accuracy: '
              f'{accu_hist_valid[epoch]:.4f}')
        
    return loss_hist_train, loss_hist_valid, \
           accu_hist_train, accu_hist_valid


In [ ]:
torch.manual_seed(1)
num_epoches = 20
hist = train(model, num_epoches, train_dl, valid_dl)

Epoch   1  train_accuracy: 0.8601, val_accuracy: 0.9572
Epoch   2  train_accuracy: 0.9642, val_accuracy: 0.9708
Epoch   3  train_accuracy: 0.9780, val_accuracy: 0.9747
Epoch   4  train_accuracy: 0.9850, val_accuracy: 0.9805
Epoch   5  train_accuracy: 0.9879, val_accuracy: 0.9757
Epoch   6  train_accuracy: 0.9896, val_accuracy: 0.9761
Epoch   7  train_accuracy: 0.9926, val_accuracy: 0.9830
Epoch   8  train_accuracy: 0.9939, val_accuracy: 0.9829


KeyboardInterrupt: ignored

let's visulaize the learning proccess!

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np
x_arr = np.arange(len(hist[0])) + 1

fig = plt.figure(figsize=(12,4))
ax = fig.add_subplot(1,2,1)
ax.plot(x_arr, hist[0],'-o', label='Train loss')
ax.plot(x_arr, hist[1],'--<', label='Validation loss')
ax.leggend(fontsize=15)
ax.set_ylable('Loss', fontsize=15)
ax.set_xlable('Epoch', fontsize=15)
ax = fig.add_subplot(1,2,2)
ax.plot(x_arr, hist[2],'-o', label='Train acc')
ax.plot(x_arr, hist[3],'--<', label='Validation acc')
ax.leggend(fontsize=15)
ax.set_ylable('Accuracy', fontsize=15)
ax.set_xlable('Epoch', fontsize=15)
plt.show()

NameError: ignored

Now, we evaluate the trained model on the test datset:

In [ ]:
pred = model(mnist_test_datset.data.unsqueeze(1) / 255.)
is_correct = (torch.argmax(pred, dim = 1) == mnist_test_datset.targets).float()
print(f'Test accuracy : {is_correct.mean():.4f}')